## Robust Query Client
This notebook is all about using the SpotifyAPI class to obtain an access token and making more complex search query requests to Spotify.

In [36]:
import requests
import base64
import datetime
import json
from urllib.parse import urlencode
from typing import Union

In [56]:
class SpotifyAPI(object):
    """
    SpotifyAPI allows for authorised api requests to 
    Spotify for artist, track, and album data.

    The SpotifyAPI class is focussed on making search 
    requests. There are several 'tiers' of request function:
        * Tier 1: base_search & get_resource: a wide range of api requests to 
        either the resource or search endpoint url.
        >>> Returns raw JSON data as dict
        
        * Tier 2: search: search Spotify for tracks, artists, albums etc
        >>> Returns raw JSON data as dict
        
        * Tier 3: get_tracks, get_artist, get_album, & get_musical_data: 
        formatted dict info returned. These are application specific,
        and may need modifying dependent on need. Alternatively, use a Tier 1 
        or Tier 2 request.
        >>> Returns formatted data as dict 
    """
    
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.perform_auth()
       
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded String.
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    
    def get_token_data(self):
        return {
        "grant_type" : "client_credentials"
        }
     
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
    
    
    def get_resource(self, lookup_id, resource_type="track", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    
    def get_artist(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="artists")
    
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="albums")
    
    
    def get_tracks(self, query: Union[str,dict]=None) -> list:
        """
        Finds track information and returns as a list of dicts.
        """
        json_data = self.search(query, "track")
        try: 
            tracks = [{'track_id': i['id'], 
              'track_name': i['name'],
              'artist': i['artists'][0]['name'],
              'track_url': i['external_urls']['spotify'],
              'image_url': i['album']['images'][0]['url']
             } for i in json_data['tracks']['items']]
        except:
            printf("json_data dict not valid")
            return [] 
        return tracks

    
    def base_search(self, query_params: str) -> dict:
        """
        Takes in url query string (typically from self.search), gets the headers
        for spotify api queries, and returns search data as dict. 
        """
        lookup_url = f"https://api.spotify.com/v1/search?{query_params}"
        headers = self.get_resource_headers()
        
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if not r.status_code in range(200, 299):
            return {}
        return r.json()
    
    
    def search(self, query: Union[str,dict]=None, search_type: str="artist", market_type: str="GB") -> str:
        """
        Takes in search query arguements and types, converts to url query string,
        and returns base_search function with that query. 
        """
        new_query=""
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            for key,value in query.items():
                new_query += f"{key}:{value} "
        query_params = urlencode({"q" : new_query, "type" : search_type.lower(), "market" : market_type})
        return self.base_search(query_params)
   

    def get_track_features(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="audio-features")
    
    
    def get_musical_data(self, track_id: str) -> dict:
        """
        Takes in a single track_id and returns a dict containing
        track_id, with formatted song key and song tempo information.
        """
        try:
            track_data = self.get_track_features(track_id)
            key = self.key_convert(track_data['key'], track_data['mode'])
            tempo = int(track_data['tempo'])
            musical_data = {"track_id": track_id,
                            "key": key,
                            "tempo": tempo}
            return musical_data
        except:
            raise Exception("No data found")
        
    
    def key_convert(self, key: int, mode: int=None) -> str:
        if key < 0 or key > 10:
            raise Exception("Invalid key value entered - bad Python joke")
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        if mode == None:
            return keys[key]
        elif mode == 1:
            return f"{keys[key]} Major"
        else:
            return f"{keys[key]} Minor"

        
    def time_signature_convert(self, time_signature: int) -> str:
        if time_signature < 0 or time_signature > 10:
            return "invalid key"
        keys = {0: "C", 1: "Db", 2: "D", 3: "Eb",
                4: "Eb", 5: "F", 6: "Gb", 7: "G",
                8: "Ab", 9: "B", 10: "Bb"}
        return keys[key]
        

### SEARCH
Let's try searching for something!

In [57]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = ""

In [62]:
# Create spotify Api instance
spotify = SpotifyAPI(client_id, client_secret)
# Search for tracks using multiple query arguements.
spotify.get_tracks({"track": "money", "artist": "pink floyd"})

https://api.spotify.com/v1/search?q=track%3Amoney+artist%3Apink+floyd+&type=track&market=GB
200


[{'track_id': '7Gx2q0ueNwvDp2BOZYGCMO',
  'track_name': 'Money - 2011 Remastered Version',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/7Gx2q0ueNwvDp2BOZYGCMO',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b27331c57b302f0e3aca46ab7561'},
 {'track_id': '6FjDEJNmx4XWYilqhTs7G9',
  'track_name': 'Money - Live',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/6FjDEJNmx4XWYilqhTs7G9',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b27369f4b7cda08f4ed73cc20474'},
 {'track_id': '77IH1EBjOSSBo30ie5XwEo',
  'track_name': 'Money - Live',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/77IH1EBjOSSBo30ie5XwEo',
  'image_url': 'https://i.scdn.co/image/ab67616d0000b2737946212e06e309d833b7999e'},
 {'track_id': '1WakuskS7SyaWnXqU4cQS7',
  'track_name': 'Money - 2001 Remastered Version',
  'artist': 'Pink Floyd',
  'track_url': 'https://open.spotify.com/track/1WakuskS7SyaWnXqU4cQS7',
  'image_url': 'https://i.scdn

In [61]:
# Get formatted musical data based on track id
spotify.get_musical_data("7Gx2q0ueNwvDp2BOZYGCMO")

{'track_id': '7Gx2q0ueNwvDp2BOZYGCMO', 'key': 'B Major', 'tempo': 124}